In [2]:
import ee
import geemap
import datetime
import ipywidgets as widgets
import pandas as pd
from IPython.display import display, HTML

# Authenticate and initialize Earth Engine
ee.Authenticate()
ee.Initialize(project='ee-priyanshusinghpatel1011')

# Define Goa boundary from assets
goa_boundary = ee.FeatureCollection('projects/ee-priyanshusinghpatel1011/assets/Goa_Boundary')

# Add a header with a blue background
header_html = """
<div style="text-align: center; background-color: #007BFF; color: white; padding: 15px; font-size: 24px; font-weight: bold; border-radius: 10px;">
    Goa Plastic Atlas
</div>
<br>
"""
display(HTML(header_html))

# Define Thresholds
NDWI_THRESHOLD = 0.1
FDI_THRESHOLD = 0.02
PI_THRESHOLD = 0.05

# Create a date picker widget with styling
date_picker = widgets.DatePicker(
    description='Select Date',
    style={'description_width': 'initial'},
    layout=widgets.Layout(width='300px')
)
display(date_picker)


# Function to compute indices
def compute_indices(image):
    ndwi = image.normalizedDifference(['B3', 'B8']).rename('NDWI')
    fdi = image.expression(
        'B8A - (B11 + ((B8A - B11) * ((833 - 665) / (1610 - 665)) * 10))',
        {
            'B8A': image.select('B8A'),
            'B11': image.select('B11')
        }).rename('FDI')
    pi = image.expression(
        '(B8) / (B8 + B4)',
        {
            'B8': image.select('B8'),
            'B4': image.select('B4')
        }).rename('PI')
    return image.addBands([ndwi, fdi, pi])
# Function to detect plastics
def detect_plastics(image):
    plastic_mask = (
        image.select('PI').gt(PI_THRESHOLD)
        .And(image.select('FDI').gt(FDI_THRESHOLD))
        .And(image.select('NDWI').gt(NDWI_THRESHOLD))
        .selfMask()  # Mask non-plastic areas
    )

    # Use only the plastic_mask for reduceToVectors
    # as the first band. It is binary (0 or 1), which is integral.
    # Remove the additional band 'PI'.
    plastic_pixels = plastic_mask.rename(['plastics'])

    # Convert raster pixels to points
    plastic_points = plastic_pixels.reduceToVectors(
        geometryType='centroid',
        reducer=ee.Reducer.countEvery(),
        scale=10,  # Sentinel-2 resolution
        geometry=goa_boundary,
        maxPixels=1e13 # increase the maxPixels to a larger value e.g 1e13
    )

    return plastic_points


# Function to fetch and display Sentinel-2 mosaic image
def display_image(date):
    date_str = date.strftime('%Y-%m-%d')
    start_date = ee.Date(date_str)
    end_date = start_date.advance(1, 'day')

    # Load and mosaic Sentinel-2 images
    image_collection = (ee.ImageCollection('COPERNICUS/S2_HARMONIZED')
                        .filterBounds(goa_boundary)
                        .filterDate(start_date, end_date)
                        .sort('CLOUDY_PIXEL_PERCENTAGE'))

    if image_collection.size().getInfo() > 0:
        global mosaic_image
        mosaic_image = image_collection.mosaic().clip(goa_boundary)
        mosaic_image = compute_indices(mosaic_image)

        # Detect plastics
        plastic_points = detect_plastics(mosaic_image)

        # Visualization parameters
        vis_params_rgb = {'bands': ['B4', 'B3', 'B2'], 'min': 0, 'max': 3000}
        vis_params_ndwi = {'bands': ['NDWI'], 'min': -1, 'max': 1, 'palette': ['blue', 'white']}
        vis_params_fdi = {'bands': ['FDI'], 'min': -0.5, 'max': 0.5, 'palette': ['black', 'yellow', 'red']}
        vis_params_pi = {'bands': ['PI'], 'min': -1, 'max': 1, 'palette': ['blue', 'white', 'red']}
        plastic_vis_params = {'color': 'cyan'}

        # Initialize map
        Map = geemap.Map()
        Map.centerObject(goa_boundary, 8)



        # Add layers to the map
        Map.addLayer(mosaic_image, vis_params_rgb, 'Sentinel-2 Mosaic RGB')
        Map.addLayer(mosaic_image.select('NDWI'), vis_params_ndwi, 'NDWI')
        Map.addLayer(mosaic_image.select('FDI'), vis_params_fdi, 'FDI')
        Map.addLayer(mosaic_image.select('PI'), vis_params_pi, 'PI')
        Map.addLayer(goa_boundary, {'color': 'red'}, 'Goa Boundary')
        Map.addLayer(plastic_points, plastic_vis_params, 'Detected Plastics')


        # Load and display live data with pop-ups (WITHOUT layer control)
        try:
            sheet_url = "https://docs.google.com/spreadsheets/d/e/2PACX-1vRlsUrejOQPqtxDJau9IMRGV2ucggDH-7FEOXbwb8Va5JgdabewAx_LcXTq_JPyPezpCAZBmE-RqfPk/pub?output=csv"
            df = pd.read_csv(sheet_url)

            # Rename columns to standard format
            df.columns = df.columns.str.lower().str.strip()

            # Ensure required columns exist
            if 'latitude' in df.columns and 'longitude' in df.columns:
                df = df.dropna(subset=['latitude', 'longitude'])  # Remove rows with NaN lat/lon

                for _, row in df.iterrows():
                    try:
                        lat, lon = float(row['latitude']), float(row['longitude'])

                        # Convert all row data into a dictionary for pop-ups
                        properties = {key: str(value) for key, value in row.to_dict().items() if pd.notna(value)}

                        # Make "Image URL" clickable
                        image_url = properties.get("image url", "")
                        if image_url.startswith("http"):  # Ensure it's a valid link
                            properties["image url"] = f'<a href="{image_url}" target="_blank" style="color: blue; text-decoration: underline;">View Image</a>'

                        # Create a popup text from the properties
                        popup_text = "<br>".join([f"<b>{key}</b>: {value}" for key, value in properties.items()])
                        popup_widget = widgets.HTML(popup_text)  # ✅ Wrap in HTML widget

                        # Add marker WITHOUT layer control
                        Map.add_marker(location=[lat, lon], popup=popup_widget)

                    except ValueError:
                        print(f"Skipping invalid row: {row}")

            else:
                print("Latitude or Longitude columns not found in the spreadsheet.")

        except Exception as e:
            print(f"Error loading live data: {e}")

        display(Map)

    else:
        print("No image available for the selected date.")

# Run when a date is selected
def on_date_change(change):
    if change['new']:
        display_image(change['new'])

# Observe the date picker
date_picker.observe(on_date_change, names='value')


DatePicker(value=None, description='Select Date', layout=Layout(width='300px'), style=DescriptionStyle(descrip…

No image available for the selected date.


Map(center=[15.300545019019628, 73.87404923647256], controls=(WidgetControl(options=['position', 'transparent_…